<a href="https://colab.research.google.com/github/not-good-keeper/sony-ssup/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y tensorflow keras tensorflow-probability onnx-tf
!pip install --upgrade tensorflow keras tensorflow-probability


Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0
Found existing installation: tensorflow-probability 0.25.0
Uninstalling tensorflow-probability-0.25.0:
  Successfully uninstalled tensorflow-probability-0.25.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 85.6 MB/s eta 0:00:00


In [2]:
!pip install onnx2tf
!pip install onnx-graphsurgeon
!pip install sng4onnx

import onnx2tf

onnx_model_path = "/content/mobilenetv3_original.onnx"
tf_model_path = "tf_model"

onnx2tf.convert(
    input_onnx_file_path=onnx_model_path,
    output_folder_path=tf_model_path,
)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.2/446.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 78.3 MB/s eta 0:00:00

Model optimizing started ============================================================
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/onnx2tf/onnx2tf.py", line 652, in convert
    result = subprocess.check_output(
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 466, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess

In [18]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

# Load CIFAR-10 dataset for calibration
ds_train = tfds.load("cifar10", split="train", as_supervised="True")

# Load TFLite converter
converter = tf.lite.TFLiteConverter.from_saved_model("tf_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Use real CIFAR-10 images for better calibration
def representative_dataset():
    for image, _ in ds_train.take(100):  # Use 100 real images
        image = tf.image.resize(image, (96, 96))
        image = tf.cast(image, dtype=tf.float32) / 255.0  # Normalize
        yield [np.expand_dims(image, axis=0).astype(np.float32)]  # Add batch dimension

converter.representative_dataset = representative_dataset

# Enable full INT8 quantization (force input/output to INT8)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # Ensure input is INT8
converter.inference_output_type = tf.int8  # Ensure output is INT8

# Convert and save the optimized INT8 model
tflite_int8_model = converter.convert()
int8_model_path = "mobilenetv3_int8_fixed.tflite"

with open(int8_model_path, "wb") as f:
    f.write(tflite_int8_model)

print("✅ Fixed INT8 model saved as:", int8_model_path)


✅ Fixed INT8 model saved as: mobilenetv3_int8_fixed.tflite


In [19]:
# Load the optimized INT8 model
interpreter = tf.lite.Interpreter(model_path="mobilenetv3_int8_fixed.tflite")
interpreter.allocate_tensors()

# Get model details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']
input_dtype = input_details[0]['dtype']
input_scale, input_zero_point = input_details[0]['quantization']

print(f"✅ Model Input Shape: {input_shape}, Data Type: {input_dtype}, Scale: {input_scale}, Zero Point: {input_zero_point}")

# Load CIFAR-10 test set
ds_test = tfds.load("cifar10", split="test", as_supervised="True")

# Resize and properly quantize images
def preprocess_image(image, label):
    image = tf.image.resize(image, (96, 96))
    image = tf.cast(image, dtype=tf.float32) / 255.0  # Normalize to [0, 1]

    # Quantize correctly for INT8 model
    image = (image / input_scale) + input_zero_point
    image = tf.clip_by_value(image, -128, 127)  # Clip to INT8 range
    image = tf.cast(image, dtype=tf.int8)  # Convert to int8

    return image, label

# Prepare the full test dataset
test_images, test_labels = [], []
for image, label in ds_test.take(10000):  # Full test set (10,000 images)
    image, label = preprocess_image(image, label)
    test_images.append(image.numpy())
    test_labels.append(label.numpy())

test_images = np.array(test_images, dtype=np.int8)
test_labels = np.array(test_labels)

# Run inference
def run_inference(interpreter, images):
    predictions = []
    input_index = input_details[0]['index']
    output_index = output_details[0]['index']

    for img in images:
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        interpreter.set_tensor(input_index, img)
        interpreter.invoke()
        output = interpreter.get_tensor(output_index)
        predictions.append(np.argmax(output))  # Get predicted class

    return np.array(predictions)

# Run inference
pred_labels = run_inference(interpreter, test_images)

# Compute accuracy
accuracy = np.mean(pred_labels == test_labels)
print(f"🎯 Fixed INT8 Model Accuracy on Full CIFAR-10 Test Set: {accuracy * 100:.2f}%")


✅ Model Input Shape: [ 1 96 96  3], Data Type: <class 'numpy.int8'>, Scale: 0.003921568859368563, Zero Point: -128
🎯 Fixed INT8 Model Accuracy on Full CIFAR-10 Test Set: 76.76%


In [20]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

# Load the CIFAR-10 dataset
ds_test = tfds.load("cifar10", split="test", as_supervised=True)

# Get input shape from TFLite model
interpreter = tf.lite.Interpreter(model_path=r"/content/tf_model/mobilenetv3_original_float32.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Model expected input shape
input_shape = input_details[0]['shape']
input_dtype = input_details[0]['dtype']
print(f"✅ Model Input Shape: {input_shape}, Data Type: {input_dtype}")

# Resize CIFAR-10 images to match model's input
IMG_SIZE = (input_shape[1], input_shape[2])  # (Height, Width)
NUM_CLASSES = 10  # CIFAR-10 has 10 classes

# Function to preprocess images
def preprocess_image(image, label):
    image = tf.image.resize(image, IMG_SIZE)  # Resize to model input size
    image = tf.cast(image, dtype=tf.float32) / 255.0  # Normalize to [0, 1]
    return image, label

# Prepare dataset for inference
test_images, test_labels = [], []
for image, label in ds_test.take(1000):  # Take 1000 samples for evaluation
    image, label = preprocess_image(image, label)
    test_images.append(image.numpy())
    test_labels.append(label.numpy())

test_images = np.array(test_images, dtype=np.float32)
test_labels = np.array(test_labels)

# Function to run inference
def run_inference(interpreter, images):
    predictions = []
    input_index = input_details[0]['index']
    output_index = output_details[0]['index']

    for img in images:
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        img = img.astype(input_dtype)  # Ensure correct dtype

        interpreter.set_tensor(input_index, img)
        interpreter.invoke()
        output = interpreter.get_tensor(output_index)

        predictions.append(np.argmax(output))  # Get predicted class index

    return np.array(predictions)

# Run inference
pred_labels = run_inference(interpreter, test_images)

# Calculate accuracy
accuracy = np.mean(pred_labels == test_labels)
print(f"🎯 INT8 Model Accuracy on 1000 CIFAR-10 test images: {accuracy * 100:.2f}%")


✅ Model Input Shape: [ 1 96 96  3], Data Type: <class 'numpy.float32'>
🎯 INT8 Model Accuracy on 1000 CIFAR-10 test images: 76.90%
